In [217]:
import pandas as pd
import json

In [218]:
with open ('../raw-files/all-party-information.json', 'r') as f:
    parties = json.load(f)['data_list']

partiesDict = dict(map(lambda x: (x['id_dga'], x), parties))

def get_partyid_for_image(dgaId):
    return partiesDict[dgaId]['id']

In [219]:
get_partyid_for_image(192) == 81

True

In [220]:
def img_path(*elements):
    return "http://202.6.26.16/images/candidate/thumb/%s/%s/%s.jpg" % elements

In [221]:
df_zone_candidates = pd.read_csv('../raw-files/candidates.csv')
df_pl_candidates = pd.read_csv('../data/partylist_candidates.csv')

In [222]:
df_zone_candidates['image'] = df_zone_candidates.apply(lambda x: img_path(x['province_id'], x['zone_number'], x['CandidateId']), axis=1)
df_zone_candidates['name'] = df_zone_candidates.apply(lambda r: '%s %s' % (r['FirstName'], r['LastName']), axis=1)
df_zone_candidates['desc'] = df_zone_candidates.apply(lambda r: 'ส.ส. เขต (%s เขต %s)' % (r['province_name'], r['zone_number']), axis=1)
df_zone_candidates['memberType'] = 'zone'

In [233]:
df_pl_candidates['name'] = df_pl_candidates.apply(lambda x: '%s %s' % (x['FirstName'], x['LastName']), axis=1)
df_pl_candidates['desc'] = df_pl_candidates.apply(lambda x: 'ส.ส. บัญชีรายชื่อ (ลำดับที่ %s)' % (x['CandidateNo']), axis=1)
df_pl_candidates['image'] = df_pl_candidates.apply(lambda x: img_path('00', get_partyid_for_image(x['PartyNumber']), x['CandidateId']), axis=1)
df_pl_candidates['memberType'] = 'partylist'
df_pl_candidates['zone_number'] = "-"
df_pl_candidates['province_name'] = "-"

In [234]:
get_partyid_for_image(192)

81

In [235]:
df_zone_candidates[df_zone_candidates.CandidateId == 'a613dcff-bc85-489f-ba1f-cbe7cdb0bbd7'].to_dict('record')

[{'Unnamed: 0': 1432,
  'CandidateId': 'a613dcff-bc85-489f-ba1f-cbe7cdb0bbd7',
  'CandidateNo': 5,
  'FirstName': 'หรรษธร',
  'LastName': 'กำปั่นทอง',
  'PartyName': 'อนาคตใหม่',
  'PartyNumber': 177,
  'Title': 'นาย',
  'Votable': True,
  'province_id': 25,
  'province_name': 'ปราจีนบุรี',
  'zone_number': 2,
  'image': 'http://202.6.26.16/images/candidate/thumb/25/2/a613dcff-bc85-489f-ba1f-cbe7cdb0bbd7.jpg',
  'name': 'หรรษธร กำปั่นทอง',
  'desc': 'ส.ส. เขต (ปราจีนบุรี เขต 2)',
  'memberType': 'zone'}]

In [236]:
df_zone_candidates.loc[0].values

array([0, '82547da2-ee9e-4e55-a9e0-9aa414956075', 1, 'ไกรสีห์',
       'รัตนเศียร', 'ชาติพัฒนา', 45, 'นาย', True, 23, 'ตราด', 1,
       'http://202.6.26.16/images/candidate/thumb/23/1/82547da2-ee9e-4e55-a9e0-9aa414956075.jpg',
       'ไกรสีห์ รัตนเศียร', 'ส.ส. เขต (ตราด เขต 1)', 'zone'], dtype=object)

In [237]:
df_pl_candidates[df_pl_candidates.CandidateId == '00d3589a-aa2b-46fc-87c8-af7f11ff4346'].values

array([[2779, '00d3589a-aa2b-46fc-87c8-af7f11ff4346', 104, 'คิว',
        'อรุโณรส', 'พลังประชารัฐ', 192, 'นาย', True, 'คิว อรุโณรส',
        'ส.ส. บัญชีรายชื่อ (ลำดับที่ 104)',
        'http://202.6.26.16/images/candidate/thumb/00/81/00d3589a-aa2b-46fc-87c8-af7f11ff4346.jpg',
        'partylist', '-', '-']], dtype=object)

In [239]:
attributes = ['name', 'desc', 'image', 'PartyName', 'memberType', 'province_name', 'zone_number']

def extract_party_politician(party):
    zone = df_zone_candidates[df_zone_candidates.PartyName == party][attributes]
    plist = df_pl_candidates[df_pl_candidates.PartyName == party][attributes]

    df_party = pd.concat([zone, plist])
    df_party.to_csv('../data/party-politicians/%s.csv' % party.strip(), index=False, header=True)
    return df_party

In [240]:
# dd = extract_party_politician('พลังประชารัฐ')
# dd

In [241]:
for p in party_names:
    extract_party_politician(p)

In [203]:
with open ('../creden-scrape.txt', 'w') as f:
    f.write('\n'.join(party_names))

In [214]:
attributes = ['CandidateId', 'name', 'CandidateNo', 'PartyName', 'province_name', 'zone_number']

all_zone_candidates = df_zone_candidates[attributes].to_dict('records')

In [215]:
# df_zone_candidates
with open('../data/all-zone-candidates.json', 'w') as f:
    json.dump(all_zone_candidates, f, ensure_ascii=False)

In [216]:
party_names

['เพื่อไทย',
 'ประชาไทย',
 'เพื่อไทยพัฒนา',
 'เศรษฐกิจใหม่',
 'เสรีรวมไทย',
 'ทางเลือกใหม่',
 'พลังคนกีฬา',
 'รักษ์ธรรม',
 'ประชาธรรมไทย',
 'เพื่อสหกรณ์ไทย',
 'มติประชา',
 'พลังสหกรณ์',
 'พลังชาติไทย',
 'พลังแผ่นดินทอง',
 'ภาคีเครือข่ายไทย',
 'ประชาภิวัฒน์',
 'เครือข่ายชาวนาแห่งประเทศไทย',
 'ประชานิยม',
 'กลาง',
 'พลังประชารัฐ',
 'เพื่อคนไทย',
 'ความหวังใหม่',
 'ไทยรุ่งเรือง',
 'พลังรัก',
 'ภราดรภาพ',
 'ยางพาราไทย',
 'พลังสังคม',
 'ชาติพันธุ์ไทย',
 'อนาคตใหม่',
 'คนงานไทย',
 'เพื่อชีวิตใหม่',
 'พลังไทสร้างชาติ',
 'กสิกรไทย',
 'พลังไทยรักไทย',
 'ภูมิใจไทย',
 'รวมใจไทย',
 'พลังศรัทธา',
 'อนาคตไทย',
 'ฐานรากไทย',
 'ไทยศรีวิไลย์',
 'แทนคุณแผ่นดิน',
 'พลังปวงชนไทย',
 'ชาติไทยพัฒนา',
 'พลังไทยดี',
 'สยามพัฒนา',
 'เพื่อธรรม',
 'เพื่อชาติ',
 'รักษ์ผืนป่าประเทศไทย',
 'พลังธรรมใหม่',
 'มหาชน',
 'ถิ่นกาขาวชาววิไล',
 'ประชาธิปไตยใหม่',
 'พลเมืองไทย',
 'เพื่อนไทย',
 'ประชาชาติ',
 'ไทรักธรรม',
 'รวมพลังประชาชาติไทย',
 'กรีน',
 'ครูไทยเพื่อประชาชน',
 'พลังประชาธิปไตย',
 'สังคมประชาธิปไตยไทย',
 'พัฒนา